In [2]:
import pandas as pd
import traceback
import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [3]:
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("(") and x.endswith(")"):
        x = x.lstrip("(")
        x = x.rstrip(")")
        float_num = float(x) * (-1)
    else:
        x = x.lstrip("(")
        x = x.rstrip(")")     
        float_num = float(x)
    return float_num

In [4]:
def year_month(str):
    substr = str[-8:-1] 
    y, m = substr.split(".")
    return y, m

In [12]:
# 보고서 접수일 기준(start ~ end), 최대 3개월 조회 가능
# kind: 정기공시(A) -> 사업보고서, 외부감사관련(F) -> 감사보고서
# 보고서 최종(final=True)
reports = dart.list(start='20140401', end='20140630', kind='F', final=True)

In [13]:
# type(reports)
reports.shape

(2966, 9)

In [14]:
# 코스피(Y), 코스닥(K) 상장회사 2013년 12월 결산법인 사업보고서
str_expr = "report_nm.str.contains('사업보고서') and corp_cls in ['Y', 'K'] and report_nm.str.contains('2013.12')"
biz_repports = reports.query(str_expr)
biz_repports.shape

(563, 9)

In [15]:
biz_repports.to_csv('D:/PythonProject/data-gatherer/dart_fs_notes/output/20140401_20140630_201312.csv')

In [20]:
import requests
USER_AGENT = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.3904.108 Safari/537.36'
rcp_no = '20140630000306'
try:
    r = requests.get(f'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}', headers={'User-Agent': USER_AGENT})
except Exception as e:
    print(e)

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [16]:
subreports = []
str_expr = "title.str.contains('5. 재무제표 주석')"
for i, row in biz_repports.iterrows():
    try:
        rpt_name = row['report_nm']
        rpt_num = row['rcept_no']        
        rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
        df_rpt = dart.sub_docs(rpt_num)         # 하루에 호출횟수 최대 100건
        rpt_data = df_rpt.query(str_expr)       # data frame
        if rpt_data.size == 0:
            subreports.append({'rpt_num' : rpt_num, 'corp_code' : row['corp_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 
                               'rpt_name' : rpt_name, 'rcept_dt' : row['rcept_dt'], 'rpt_url' : rpt_url, 'rpt_title' : None, 'rpt_notes_url' : None, 'result' : 'No Financial Statement Notes'})                 
        else:
            rpt_title = rpt_data.iloc[0]['title']
            rpt_notes_url = rpt_data.iloc[0]['url']
            subreports.append({'rpt_num' : rpt_num, 'corp_code' : row['corp_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 
                               'rpt_name' : rpt_name, 'rcept_dt' : row['rcept_dt'], 'rpt_url' : rpt_url, 'rpt_title' : rpt_title, 'rpt_notes_url' : rpt_notes_url, 'result' : None}) 
    except Exception as e:
        print(row['rcept_no'])
        print(row['stock_code'])
        print(row['report_nm'])
        print('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'])
        err_msg = traceback.format_exc()
        print(err_msg)

20140630000306
199800
사업보고서 (2013.12)
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140630000306
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py",

In [33]:
len(subreports)

0

In [45]:
data_list = []
for r in subreports:
    try:
        val_a = val_b = None
        url = r['rpt_notes_url']
        if url == None:
            data_list.append({'corp_code' : r['corp_code'], 'corp_cls' : r['corp_cls'], 'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 
                              'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 'rcept_dt' : r['rcept_dt'], 
                              'val_a' : None, 'val_b' : None, 'result' : 'No Financial Statement Notes'})
        else:
            doc_data = pd.read_html(url)
            for df in doc_data:
                row = df.shape[0]
                for i in range(0, row):
                    item = str(df.iloc[i,0])
                    if item.__contains__('확정급여채무의 현재가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                        str_a = str(df.iloc[i,1])
                        if str_a == "-": val_a = None
                        else: val_a = str2num(str_a)
                    elif item.__contains__('사외적립자산의 공정가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
                        str_b = str(df.iloc[i,1])
                        if str_b == "-": val_b = None
                        else: val_b = str2num(str_b)
                    else:
                        continue    
                if (val_a != None) and (val_b != None):
                    break          
            data_list.append({'corp_code' : r['corp_code'], 'corp_cls' : r['corp_cls'], 'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 
                              'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 'rcept_dt' : r['rcept_dt'], 
                              'val_a' : val_a, 'val_b' : val_b, 'result' : 'OK'})
    except ValueError as err:
        print(r['rpt_num'])
        # print(r['rpt_url'])
        print(r['rpt_notes_url'])
        # print(err)
        data_list.append({'corp_code' : r['corp_code'], 'corp_cls' : r['corp_cls'], 'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 
                          'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 'rcept_dt' : r['rcept_dt'], 
                          'val_a' : val_a, 'val_b' : val_b, 'result' : str(err)})        
    except Exception as e:
        print(r['rpt_num'])
        # print(r['rpt_url'])
        print(r['rpt_notes_url'])
        err_msg = traceback.format_exc()
        print(err_msg)

20170428002043
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428002043&dcmNo=5617894&eleId=16&offset=555569&length=130&dtd=dart3.xsd
20170428001871
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001871&dcmNo=5617406&eleId=16&offset=608249&length=216&dtd=dart3.xsd
20170428001837
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001837&dcmNo=5617323&eleId=16&offset=572391&length=211&dtd=dart3.xsd
20170428001583
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001583&dcmNo=5616748&eleId=16&offset=741910&length=136&dtd=dart3.xsd
20170428001099
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428001099&dcmNo=5616006&eleId=16&offset=799597&length=196&dtd=dart3.xsd
20170427000709
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170427000709&dcmNo=5612910&eleId=16&offset=636285&length=229&dtd=dart3.xsd
20170418000281
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170418000281&dcmNo=5600517&eleId=16&offset=56844&length=121&dtd=dart3.xsd
20170410002581
http://dart.fss.or.kr/repor

In [46]:
len(data_list)

99

In [47]:
df_reports = pd.DataFrame(data_list)

In [48]:
df_reports.shape

(99, 7)

In [49]:
df_reports['result'].value_counts()

result
OK                              79
No Financial Statement Notes    11
No tables found                  9
Name: count, dtype: int64

In [50]:
df_reports[df_reports['result'] == 'OK']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
3,알바이오,003190,20170428001738,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
4,세종,,20170428001652,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
6,이에스산업,241510,20170428001415,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
8,듀켐바이오,176750,20170428001089,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
9,에스텍시스템,,20170428000888,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
...,...,...,...,...,...,...,...
94,크린앤사이언스,045520,20170403000445,[기재정정]사업보고서 (2016.12),3885459.0,-3841856.0,OK
95,피엠티,147760,20170403000323,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
96,퓨쳐켐,220100,20170403000225,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
97,교보증권,030610,20170403000043,[기재정정]사업보고서 (2016.12),30458251.0,-30339335.0,OK


In [51]:
df_reports[df_reports['result'] == 'No Financial Statement Notes']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
11,비앤에스미디어,156170,20170428000153,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
14,큐렉소,060280,20170426000359,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
16,삼일,032280,20170426000246,[기재정정]사업보고서 (2012.12),NaN,NaN,No Financial Statement Notes
18,우진,105840,20170421000696,[기재정정]사업보고서 (2013.12),NaN,NaN,No Financial Statement Notes
19,우양에이치씨,101970,20170421000455,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
25,한화,000880,20170419000057,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
45,한창산업,079170,20170411000156,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
47,넵튠,217270,20170410002533,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
57,롯데손해보험,000400,20170406002062,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes
59,농업회사법인합천유통,,20170406001175,[첨부정정]사업보고서 (2016.12),NaN,NaN,No Financial Statement Notes


In [52]:
df_reports[df_reports['result'] == 'No tables found']

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
0,이스트아시아홀딩스,900110,20170428002043,사업보고서 (2016.12),NaN,NaN,No tables found
1,오가닉티코스메틱,900300,20170428001871,사업보고서 (2016.12),NaN,NaN,No tables found
2,차이나그레이트스타인터내셔널리미티드,900040,20170428001837,사업보고서 (2016.12),NaN,NaN,No tables found
5,로스웰,900260,20170428001583,사업보고서 (2016.12),NaN,NaN,No tables found
7,엘브이엠씨,900140,20170428001099,사업보고서 (2016.12),NaN,NaN,No tables found
13,골든센츄리,900280,20170427000709,사업보고서 (2016.12),NaN,NaN,No tables found
28,베어리버,,20170418000281,사업보고서 (2016.12),NaN,NaN,No tables found
46,뉴아세아,013340,20170410002581,사업보고서 (2016.12),NaN,NaN,No tables found
61,마이크로사이언스테크,,20170406000945,사업보고서 (2016.12),NaN,NaN,No tables found


In [20]:
df_reports.dtypes

corp_name      object
stock_code     object
rpt_num        object
rpt_name       object
val_a         float64
val_b         float64
result         object
dtype: object

In [34]:
df_reports

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b,result
0,이스트아시아홀딩스,900110,20170428002043,사업보고서 (2016.12),NaN,NaN,No tables found
1,오가닉티코스메틱,900300,20170428001871,사업보고서 (2016.12),NaN,NaN,No tables found
2,차이나그레이트스타인터내셔널리미티드,900040,20170428001837,사업보고서 (2016.12),NaN,NaN,No tables found
3,알바이오,003190,20170428001738,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
4,세종,,20170428001652,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
...,...,...,...,...,...,...,...
94,크린앤사이언스,045520,20170403000445,[기재정정]사업보고서 (2016.12),3885459.0,-3841856.0,OK
95,피엠티,147760,20170403000323,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
96,퓨쳐켐,220100,20170403000225,[기재정정]사업보고서 (2016.12),NaN,NaN,OK
97,교보증권,030610,20170403000043,[기재정정]사업보고서 (2016.12),30458251.0,-30339335.0,OK


In [ ]:
df_reports['year'], df_reports['month'] = zip(*df_reports['rpt_name'].apply(year_month))

In [43]:
df_dtypes = {'corp_code': 'string', 'corp_name': 'string', 'stock_code': 'string', 'modify_date': 'string'}
df_corp_codes = pd.read_csv('D:/PythonProject/data-gatherer/dart/output/corp_codes.csv', dtype=df_dtypes)

In [44]:
df_corp_codes.dtypes

corp_code      string[python]
corp_name      string[python]
stock_code     string[python]
modify_date    string[python]
dtype: object

In [46]:
df_corp_codes

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,<NA>,20170630
1,00434456,일산약품,<NA>,20170630
2,00430964,굿앤엘에스,<NA>,20170630
3,00432403,한라판지,<NA>,20170630
4,00388953,크레디피아제이십오차유동화전문회사,<NA>,20170630
...,...,...,...,...
103827,01525205,정담유통,<NA>,20230228
103828,00646510,미래에셋파트너스사호사모투자전문회사,<NA>,20230228
103829,01184822,미래에셋파트너스9호사모투자,<NA>,20230228
103830,00755252,시니안,<NA>,20230228


In [ ]:
df_all = pd.merge(df_reports, df_corp_codes, how='left', left_on='corp_code', right_on='corp_code')

In [ ]:
df_all.head(5)

In [53]:
df_all.to_csv('D:/PythonProject/data-gatherer/dart_fs_notes/output/result_1m.csv')